<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import requests

url = "https://www.gutenberg.org/files/100/100-0.txt"
r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text

In [2]:
data[:1000]

'\r\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever.  You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org.  If you are not located in the United States, you’ll\r\nhave to check the laws of the country where you are located before using\r\nthis ebook.\r\n\r\n\r\nTitle: The Complete Works of William Shakespeare\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease Date: January 1994 [EBook #100]\r\nLast Updated: November 7, 2019\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***\r\n\r\n\r\n\r\n\r\nThe Complete Works of William Shakespeare\r\n\r\n\r\n\r\nby William Shakespeare\r\n\r\

In [3]:
data[-25000:]

'his ears a heavy tale,\r\nAs if they heard the woeful words she told;\r\nShe lifts the coffer-lids that close his eyes,\r\nWhere lo, two lamps burnt out in darkness lies.\r\n\r\nTwo glasses where herself herself beheld            1129\r\nA thousand times, and now no more reflect;\r\nTheir virtue lost, wherein they late excell’d,\r\nAnd every beauty robb’d of his effect.              1132\r\n  “Wonder of time,” quoth she, “this is my spite,\r\n  That thou being dead, the day should yet be light.\r\n\r\n“Since thou art dead, lo here I prophesy,\r\nSorrow on love hereafter shall attend:              1136\r\nIt shall be waited on with jealousy,\r\nFind sweet beginning, but unsavoury end;\r\n  Ne’er settled equally, but high or low,\r\n  That all love’s pleasure shall not match his woe.\r\n\r\n“It shall be fickle, false and full of fraud,       1141\r\nBud, and be blasted in a breathing while;\r\nThe bottom poison, and the top o’erstraw’d\r\nWith sweets that shall the truest sight beguile.

In [4]:
len(data)

5740053

In [5]:
# clean up the text:
# want everything between 'Contents' and 'finis'
one_string = (str(data).split("Contents\r\n\r\n\r\n\r\n", 1)[1]).split("FINIS\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r", 1)[0]
len(one_string)

5717561

In [6]:
one_string[-1000:]

' the stalk, and in the breach appears\r\n  Green-dropping sap, which she compares to tears.\r\n\r\n“Poor flower,” quoth she, “this was thy father’s guise,\r\nSweet issue of a more sweet-smelling sire,\r\nFor every little grief to wet his eyes,\r\nTo grow unto himself was his desire,                1180\r\n  And so ’tis thine; but know, it is as good\r\n  To wither in my breast as in his blood.\r\n\r\n“Here was thy father’s bed, here in my breast;\r\nThou art the next of blood, and ’tis thy right:     1184\r\nLo in this hollow cradle take thy rest,\r\nMy throbbing heart shall rock thee day and night:\r\n  There shall not be one minute in an hour\r\n  Wherein I will not kiss my sweet love’s flower.”\r\n\r\nThus weary of the world, away she hies,             1189\r\nAnd yokes her silver doves; by whose swift aid\r\nTheir mistress mounted through the empty skies,\r\nIn her light chariot quickly is convey’d;           1192\r\n  Holding their course to Paphos, where their queen\r\n  Means t

In [7]:
one_string[:1000]

'               THE SONNETS\r\n\r\n               ALL’S WELL THAT ENDS WELL\r\n\r\n               THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\n               AS YOU LIKE IT\r\n\r\n               THE COMEDY OF ERRORS\r\n\r\n               THE TRAGEDY OF CORIOLANUS\r\n\r\n               CYMBELINE\r\n\r\n               THE TRAGEDY OF HAMLET, PRINCE OF DENMARK\r\n\r\n               THE FIRST PART OF KING HENRY THE FOURTH\r\n\r\n               THE SECOND PART OF KING HENRY THE FOURTH\r\n\r\n               THE LIFE OF KING HENRY THE FIFTH\r\n\r\n               THE FIRST PART OF HENRY THE SIXTH\r\n\r\n               THE SECOND PART OF KING HENRY THE SIXTH\r\n\r\n               THE THIRD PART OF KING HENRY THE SIXTH\r\n\r\n               KING HENRY THE EIGHTH\r\n\r\n               KING JOHN\r\n\r\n               THE TRAGEDY OF JULIUS CAESAR\r\n\r\n               THE TRAGEDY OF KING LEAR\r\n\r\n               LOVE’S LABOUR’S LOST\r\n\r\n               THE TRAGEDY OF MACBETH\r\n\r\n               

In [8]:
# encode data as chars:
chars = list(set(one_string))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [9]:
len(chars)

102

In [10]:
# create sequence data:
maxlen = 40
step = 5

encoded = [char_int[c] for c in one_string]

sequences = []
next_chars = []

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 1143505


In [11]:
# specify x & y:
import numpy as np
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [12]:
x.shape, y.shape

((1143505, 40, 102), (1143505, 102))

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
# build model:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import numpy as np
import random
import sys
import os


model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [16]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(one_string) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = one_string[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [17]:
model.fit(x, y,
          batch_size=64,
          epochs=5,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1143488/1143505 [============================>.] - ETA: 0s - loss: 2.0115
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ing.
  KING HENRY. Henry the Fourth by "
ing.
  KING HENRY. Henry the Fourth by the conters the sone and the sone and the sone and the sond.

                                                                                                                                                                                                                                                                                                                                                
----- diversity: 0.5
----- Generating with seed: "ing.
  KING HENRY. Henry the Fourth by "
ing.
  KING HENRY. Henry the Fourth by his a word with, what when the gray,
    So pone the sirce handels of the bester.

                                                                                         

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN